In [ ]:
# Import Libraries
import pandas as pd
import rhutils as ru
import gspread_pandas as gs
import numpy as np
import datetime as dt
import pytz, time, datetime
from datetime import datetime, timedelta
from datetime import date
import re
import os
import pytz
from datetime import datetime 

os.chdir('/opt/app-root/src/americas-notebooks/notebooks/1_GLOBAL')
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows=100
pd.options.display.max_columns=500

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
# Import Libraries for Google Drive Upload
from apiclient import discovery
from google.oauth2 import service_account
from googleapiclient.http import MediaFileUpload
import io

SCOPES = ['https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = 'service_client_id.json'
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
DRIVE = discovery.build('drive', 'v3', credentials=credentials, cache_discovery=False)

# Input Google Drive Folder ID
# https://drive.google.com/drive/folders/1ev1WLB22yfvX9Fzd-26ZZ-H3blVgcIZn
gid = '1ev1WLB22yfvX9Fzd-26ZZ-H3blVgcIZn'
gid2 = '1kPr77CCbLISxS2MIYiFaDDFwS4jKwyaH'
gid3 = '1fqcoSbDnLurbSurgeayNusXkZKHVIa30'

In [ ]:
# Autodate Check
est = pytz.timezone('America/New_York')
today = datetime.now(est).strftime("%Y-%m-%d")

In [ ]:
os.chdir('/opt/app-root/src/americas-notebooks/notebooks/PROJECTS/Commercial_MVP/Bloom_MVP_v2')

In [ ]:
File = 'Commercial_MVP_File'
df_1 = pd.read_csv(f'{File}_{today}.csv')
df_2 = pd.read_csv('MVP_Planner_Output_Tab1.csv')
df_1.shape, df_2.shape

In [ ]:
df_1.head()

In [ ]:
sel_measures = [15, 17, 18, 57, 60,  6,  8, 29, 30, 31]
df_1 = df_1[df_1['measure'].isin(sel_measures)]
df_1.shape

In [ ]:
df_1['measure'] = df_1['measure'].astype(str)
df_2['Measure'] = df_2['Measure'].astype(str).replace('\.0', '', regex=True)
df_2.head()

In [ ]:
for col in df_1.select_dtypes(include=['object']).columns.tolist():
    df_1[col] = df_1[col].str.upper()

for col in df_2.select_dtypes(include=['object']).columns.tolist():
    new_col_name = col + '_forLookup'
    df_2[new_col_name] = df_2[col].str.upper()
    
df_2['Product_forLookup'] = df_2['Product_forLookup'].str.upper().str.replace(' ', '_')
df_1.shape, df_2.shape

In [ ]:
#df_1.head()

In [ ]:
#df_2.head()

In [ ]:
'''
# one particular line for debugging
df_1[(df_1['reporting_geo'] == 'APAC') & 
     (df_1['reporting_region'] == 'ANZ') & 
     (df_1['reporting_subregion'] == 'ANZ') & 
     (df_1['reporting_subsegment'] == 'TERRITORY') &  
     (df_1['product_family_treated'] == 'OTHER_PRODUCTS_SYB') & 
    (df_1['measure'] == '15')]
'''

In [ ]:
'''
# one particular line for debugging
df_2[(df_2['Geo'] == 'APAC') & 
     (df_2['Region'] == 'ANZ') & 
     (df_2['SubRegion'] == 'ANZ') & 
     (df_2['SubSegment'] == 'TERRITORY') &  
     (df_2['Product'] == 'OTHER_PRODUCTS_SYB') & 
    (df_2['Measure'] == '15') ]
'''

In [ ]:
merged_df = pd.merge(df_2, 
                     df_1, 
                     left_on=['Geo_forLookup', 'Region_forLookup', 'SubRegion_forLookup', 'SubSegment_forLookup', 
                              'Product_forLookup', 'Measure_forLookup'], 
                     right_on=['reporting_geo', 'reporting_region', 'reporting_subregion', 'reporting_subsegment', 
                              'product_family_treated', 'measure'], 
                     how='left')

#merged_df[merged_df['value'].notnull()]
df_2['value_amd'] = merged_df['syb']
df_2.reset_index(drop=True, inplace=True)
df_2 = df_2.reindex(columns=['Geo', 
                             'Region', 
                             'SubRegion',
                             'SubSegment',
                             'Product',
                             'Measure',
                             'Geo_forLookup',
                             'Region_forLookup', 
                             'SubRegion_forLookup', 
                             'SubSegment_forLookup', 
                              'Product_forLookup', 
                             'Measure_forLookup',
                             'value',                             
                             'value_amd'])
df_2.head()

In [ ]:
'''
# one particular line for debugging
df_2[(df_2['Geo_forLookup'] == 'APAC')& 
     (df_2['Region_forLookup'] == 'ANZ') & 
     (df_2['SubRegion_forLookup'] == 'ANZ') & 
     (df_2['SubSegment_forLookup'] == 'TERRITORY')&  
     (df_2['Product_forLookup'] == 'OTHER_PRODUCTS_SYB') & 
    (df_2['Measure_forLookup'] == '15')]
'''

In [ ]:
'''
# one particular line for debugging
merged_df[(merged_df['reporting_geo'] == 'APAC') & 
          (merged_df['reporting_region'] == 'ANZ') & 
          (merged_df['reporting_subregion'] == 'ANZ') & 
          (merged_df['reporting_subsegment'] == 'TERRITORY') &  
          (merged_df['product_family_treated'] == 'OTHER_PRODUCTS_SYB') & 
          (merged_df['measure'] == '15')]
'''

In [ ]:
sel_measures = [str(x) for x in sel_measures]
df_2['final_value'] = df_2.apply(lambda x: x['value_amd'] if x['Measure'] in sel_measures else x['value'], axis=1)

In [ ]:
'''
# one particular line for debugging
df_2[(df_2['Geo_forLookup'] == 'APAC')& 
     (df_2['Region_forLookup'] == 'ANZ') & 
     (df_2['SubRegion_forLookup'] == 'ANZ') & 
     (df_2['SubSegment_forLookup'] == 'TERRITORY')&  
     (df_2['Product_forLookup'] == 'OTHER_PRODUCTS_SYB') & 
    (df_2['Measure_forLookup'] == '15')]
'''

In [ ]:
# cleaning of columns after calculating the value
df_2 = df_2.drop(columns=['value', 'value_amd', 'Geo_forLookup', 'Region_forLookup', 'SubRegion_forLookup', 
                          'SubSegment_forLookup', 'Product_forLookup', 'Measure_forLookup'])
df_2 = df_2.rename(columns={'final_value': 'value'})

In [ ]:
df_2['value'] = df_2['value'].replace('-', np.nan)
df_2['value'] = df_2['value'].replace('.', np.nan)
df_2['value'] = df_2['value'].astype(float)
df_2.head()

## Writing the table

In [ ]:
# To Redshift
ru.to_redshift(df_2, 
               table_name="blt_commercial_mvp_partial_filled", 
               schema="bf_ard", 
               if_exists = "replace", 
               group='bf_ard_ro')

In [ ]:
'''
# upload of a query result WITHOUT saving first the CSV - space is a scarse resource!!

from io import BytesIO, StringIO
from googleapiclient.http import MediaFileUpload, MediaIoBaseUpload

# Commercial MVP File
File = 'blt_commercial_mvp_partial_filled'

query = """SELECT * FROM bf_ard.blt_commercial_mvp_partial_filled;"""
hb = ru.query_RS(query)

# Convert DataFrame to string buffer
csv_buffer = StringIO()
hb.to_csv(csv_buffer, index=False)

# Convert string buffer to bytes
csv_bytes = csv_buffer.getvalue().encode('utf-8')

# Gdrive Upload
media = MediaIoBaseUpload(BytesIO(csv_bytes), mimetype='text/csv',chunksize=1024*1024, resumable=True)
file_metadata = {'name': f'{File}_{today}.csv', 'parents': [gid]}
request = DRIVE.files().create(body=file_metadata, media_body=media, supportsAllDrives=True)
response = None
while response is None:
    status, response = request.next_chunk()
    if status:
        print("Uploaded %d%%." % int(status.progress() * 100))
print("Upload Complete!")
'''